In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
from model_arch.run_train import *
from utils.step_sample import create_named_schedule_sampler
from model_arch.train import TrainLoop
from utils.data import load_data_text
from model_arch.tokenizer import load_tokenizer, load_model_emb
from model_arch.sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch
from utils import dist_util
from functools import partial
import pickle
import random
from datetime import datetime

In [ ]:
dist_util.clear_cache()

In [4]:
lr=0.0001
batch_size=30
val_batch_size=30
microbatch=10
epochs=30_000
eval_interval=10
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=2500
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.1
seed=10275679
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [5]:
regular_data_dir='data'
data_player_dir='data/with_player'
comedies_data_dir='data/comedies_only'

# set the data directory
data_dir=regular_data_dir

In [6]:
set_seed(seed)

2024-03-10 07:29:18.208833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-10 07:29:18.208936: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-10 07:29:18.210990: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-10 07:29:18.225627: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-10 07:29:20.905592: W tensorflow/compiler/tf2

In [7]:
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [8]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [9]:
model_weight

Embedding(30268, 128)

In [10]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size
vocab_size

30268

In [11]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

val = load_data_text(
        batch_size=val_batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        split='valid',
        model_emb=model_weight, # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading form the TRAIN set...
### Data samples...
 ['this is the very false gallop of verses why do you infect yourself with them?', 'and fair men call for grace. aaron will have his soul black like his face.'] ['peace, you dull fool! i found them on a tree.', 'o, here i lift this one hand up to heaven, and bow this feeble ruin to the earth if any power pities wretched tears, to that i call! what, wilt thou kneel with me? do, then, dear heart,']
RAM used: 1398.54 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 32531
})
RAM used: 1417.53 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/32531 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 32531
})
### tokenized_datasets...example [2, 138, 121, 78, 476, 880, 9871, 94, 5833, 329, 144, 89, 2960, 955, 131, 266, 22, 3]
RAM used: 1459.36 MB


merge and mask:   0%|          | 0/32531 [00:00<?, ? examples/s]

RAM used: 1493.67 MB


...

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 8123
})
### tokenized_datasets...example [2, 714, 37, 9, 749, 243, 23, 550, 85, 317, 31, 241, 125, 100, 208, 10, 6206, 10, 136, 469, 23187, 180, 646, 37, 9, 579, 3437, 10, 85, 1044, 549, 31, 9, 78, 5789, 237, 571, 295, 37, 9, 78, 14784, 12, 3]
RAM used: 1534.86 MB


merge and mask:   0%|          | 0/8123 [00:00<?, ? examples/s]

RAM used: 1551.21 MB


padding:   0%|          | 0/8123 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 8123
}) padded dataset
RAM used: 1560.91 MB
RAM used: 1560.91 MB


In [12]:
model, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        diffusion_steps,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

In [13]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

91192508

In [14]:
# from torch import nn
# from transformers import BertConfig

# config = BertConfig.from_pretrained("bert-base-uncased")

# for layer in model.input_transformers.layer[-1:]:
#     for module in layer.modules():
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=config.initializer_range)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)

In [15]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

model.to(dist_util.dev())

train_loss, val_loss = TrainLoop(
                            model=model,
                            diffusion=diffusion,
                            data=data,
                            batch_size=batch_size,
                            microbatch=microbatch,
                            lr=lr,
                            ema_rate=ema_rate,
                            schedule_sampler=schedule_sampler,
                            weight_decay=weight_decay,
                            epochs=epochs,
                            eval_data=val,
                            eval_interval=eval_interval,
                            warm_up_steps=500,
                            use_llrd=True,
                            llrd_rate=0.9
                        ).run_loop()

dt = datetime.now().strftime("%m%d")
pickle.dump(model, open(f"models/{dt}/final_model_df{diffusion_steps}.pkl", 'wb'))



======== Using Layer-wise Learning Rate Decay with AdamW ========


name: word_embedding.weight, lr: 0.0001
name: lm_head.bias, lr: 0.0001
name: time_embed.0.weight, lr: 0.0001
name: time_embed.0.bias, lr: 0.0001
name: time_embed.2.weight, lr: 0.0001
name: time_embed.2.bias, lr: 0.0001
name: input_up_proj.0.weight, lr: 0.0001
name: input_up_proj.0.bias, lr: 0.0001
name: input_up_proj.2.weight, lr: 0.0001
name: input_up_proj.2.bias, lr: 0.0001
name: input_transformers.layer.0.attention.self.query.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.query.bias, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.key.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.key.bias, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.value.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.value.bias, lr: 0.00011111111111111112
name: input_transformers.la



======== Training starts now ========


Epoch 0/30000 Training Loss: 1.003659725189209
Epoch 0/30000 Validation Loss: 1.0104681253433228
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=1.0104681253433228<=============
Epoch 1/30000 Training Loss: 0.9923286437988281
Epoch 2/30000 Training Loss: 1.0045844316482544
Epoch 3/30000 Training Loss: 1.002435564994812
Epoch 4/30000 Training Loss: 0.9960399270057678
Epoch 5/30000 Training Loss: 0.9771733283996582
Epoch 6/30000 Training Loss: 0.9623767733573914
Epoch 7/30000 Training Loss: 0.9455592632293701
Epoch 8/30000 Training Loss: 0.9263091087341309
Epoch 9/30000 Training Loss: 0.9202634692192078
Epoch 10/30000 Training Loss: 0.9158298373222351
Epoch 10/30000 Validation Loss: 0.8674793839454651
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.8674793839454651<=============
Epoch 11/30000 Training Loss: 0.8744320869445801
Epoch 12/30000 Tr

Epoch 128/30000 Training Loss: 0.345733642578125
Epoch 129/30000 Training Loss: 0.38116654753685
Epoch 130/30000 Training Loss: 0.3444702625274658
Epoch 130/30000 Validation Loss: 0.4096357524394989
Epoch 131/30000 Training Loss: 0.435497522354126
Epoch 132/30000 Training Loss: 0.3981006145477295
Epoch 133/30000 Training Loss: 0.406393438577652
Epoch 134/30000 Training Loss: 0.3865523338317871
Epoch 135/30000 Training Loss: 0.3896009027957916
Epoch 136/30000 Training Loss: 0.37286362051963806
Epoch 137/30000 Training Loss: 0.4284806549549103
Epoch 138/30000 Training Loss: 0.36519357562065125
Epoch 139/30000 Training Loss: 0.35374295711517334
Epoch 140/30000 Training Loss: 0.4343454837799072
Epoch 140/30000 Validation Loss: 0.37261509895324707
Epoch 141/30000 Training Loss: 0.3805222511291504
Epoch 142/30000 Training Loss: 0.37285280227661133
Epoch 143/30000 Training Loss: 0.35261037945747375
Epoch 144/30000 Training Loss: 0.3146049976348877
Epoch 145/30000 Training Loss: 0.367417216300

Epoch 263/30000 Training Loss: 0.23279626667499542
Epoch 264/30000 Training Loss: 0.23650677502155304
Epoch 265/30000 Training Loss: 0.22189448773860931
Epoch 266/30000 Training Loss: 0.24423235654830933
Epoch 267/30000 Training Loss: 0.23631392419338226
Epoch 268/30000 Training Loss: 0.21535147726535797
Epoch 269/30000 Training Loss: 0.22471962869167328
Epoch 270/30000 Training Loss: 0.20850960910320282
Epoch 270/30000 Validation Loss: 0.21432721614837646
Epoch 271/30000 Training Loss: 0.19839288294315338
Epoch 272/30000 Training Loss: 0.20996077358722687
Epoch 273/30000 Training Loss: 0.21007150411605835
Epoch 274/30000 Training Loss: 0.20856647193431854
Epoch 275/30000 Training Loss: 0.19835250079631805
Epoch 276/30000 Training Loss: 0.21460212767124176
Epoch 277/30000 Training Loss: 0.2239631861448288
Epoch 278/30000 Training Loss: 0.23002423346042633
Epoch 279/30000 Training Loss: 0.23247285187244415
Epoch 280/30000 Training Loss: 0.20708304643630981
Epoch 280/30000 Validation Los

Epoch 401/30000 Training Loss: 0.21176010370254517
Epoch 402/30000 Training Loss: 0.19729918241500854
Epoch 403/30000 Training Loss: 0.21428458392620087
Epoch 404/30000 Training Loss: 0.17617078125476837
Epoch 405/30000 Training Loss: 0.16578342020511627
Epoch 406/30000 Training Loss: 0.17628608644008636
Epoch 407/30000 Training Loss: 0.1341099590063095
Epoch 408/30000 Training Loss: 0.20161576569080353
Epoch 409/30000 Training Loss: 0.15495429933071136
Epoch 410/30000 Training Loss: 0.16808103024959564
Epoch 410/30000 Validation Loss: 0.15759938955307007
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.15759938955307007<=============
Epoch 411/30000 Training Loss: 0.1898844838142395
Epoch 412/30000 Training Loss: 0.16599760949611664
Epoch 413/30000 Training Loss: 0.1914958953857422
Epoch 414/30000 Training Loss: 0.1489611268043518
Epoch 415/30000 Training Loss: 0.15740834176540375
Epoch 416/30000 Training Loss: 0.19764012098312378
Ep

...

Epoch 29411/30000 Training Loss: 0.08844638615846634
Epoch 29412/30000 Training Loss: 0.07605748623609543
Epoch 29413/30000 Training Loss: 0.07031793147325516
Epoch 29414/30000 Training Loss: 0.07325994223356247
Epoch 29415/30000 Training Loss: 0.0705869197845459
Epoch 29416/30000 Training Loss: 0.06544135510921478
Epoch 29417/30000 Training Loss: 0.06272672861814499
Epoch 29418/30000 Training Loss: 0.08519405871629715
Epoch 29419/30000 Training Loss: 0.062292929738759995
Epoch 29420/30000 Training Loss: 0.08137050271034241
Epoch 29420/30000 Validation Loss: 0.06941104680299759
Epoch 29421/30000 Training Loss: 0.053806859999895096
Epoch 29422/30000 Training Loss: 0.08170471340417862
Epoch 29423/30000 Training Loss: 0.05778460204601288
Epoch 29424/30000 Training Loss: 0.05662136152386665
Epoch 29425/30000 Training Loss: 0.06619114428758621
Epoch 29426/30000 Training Loss: 0.0807393491268158
Epoch 29427/30000 Training Loss: 0.07748473435640335
Epoch 29428/30000 Training Loss: 0.069232262

Epoch 29552/30000 Training Loss: 0.059606846421957016
Epoch 29553/30000 Training Loss: 0.08076890558004379
Epoch 29554/30000 Training Loss: 0.05545759201049805
Epoch 29555/30000 Training Loss: 0.05834592506289482
Epoch 29556/30000 Training Loss: 0.05754968523979187
Epoch 29557/30000 Training Loss: 0.06709971278905869
Epoch 29558/30000 Training Loss: 0.06664503365755081
Epoch 29559/30000 Training Loss: 0.06685107201337814
Epoch 29560/30000 Training Loss: 0.06898230314254761
Epoch 29560/30000 Validation Loss: 0.056900691241025925
Epoch 29561/30000 Training Loss: 0.0747489482164383
Epoch 29562/30000 Training Loss: 0.08035110682249069
Epoch 29563/30000 Training Loss: 0.07297917455434799
Epoch 29564/30000 Training Loss: 0.06404594331979752
Epoch 29565/30000 Training Loss: 0.07064498215913773
Epoch 29566/30000 Training Loss: 0.07631554454565048
Epoch 29567/30000 Training Loss: 0.06114937737584114
Epoch 29568/30000 Training Loss: 0.0922192633152008
Epoch 29569/30000 Training Loss: 0.063927672

Epoch 29693/30000 Training Loss: 0.05481107905507088
Epoch 29694/30000 Training Loss: 0.07411172986030579
Epoch 29695/30000 Training Loss: 0.06775624305009842
Epoch 29696/30000 Training Loss: 0.05792106315493584
Epoch 29697/30000 Training Loss: 0.0665382444858551
Epoch 29698/30000 Training Loss: 0.059959590435028076
Epoch 29699/30000 Training Loss: 0.08537384122610092
Epoch 29700/30000 Training Loss: 0.0601501502096653
Epoch 29700/30000 Validation Loss: 0.05788363888859749
Epoch 29701/30000 Training Loss: 0.06944718956947327
Epoch 29702/30000 Training Loss: 0.06732375174760818
Epoch 29703/30000 Training Loss: 0.07497125118970871
Epoch 29704/30000 Training Loss: 0.06444492191076279
Epoch 29705/30000 Training Loss: 0.07348491996526718
Epoch 29706/30000 Training Loss: 0.08090364933013916
Epoch 29707/30000 Training Loss: 0.06191723421216011
Epoch 29708/30000 Training Loss: 0.07836636155843735
Epoch 29709/30000 Training Loss: 0.07100367546081543
Epoch 29710/30000 Training Loss: 0.0685197338

Epoch 29834/30000 Training Loss: 0.07264196127653122
Epoch 29835/30000 Training Loss: 0.06639423966407776
Epoch 29836/30000 Training Loss: 0.06093986704945564
Epoch 29837/30000 Training Loss: 0.06779066473245621
Epoch 29838/30000 Training Loss: 0.06769602000713348
Epoch 29839/30000 Training Loss: 0.06103569641709328
Epoch 29840/30000 Training Loss: 0.07499050348997116
Epoch 29840/30000 Validation Loss: 0.07180424779653549
Epoch 29841/30000 Training Loss: 0.09301304817199707
Epoch 29842/30000 Training Loss: 0.06276995688676834
Epoch 29843/30000 Training Loss: 0.06812797486782074
Epoch 29844/30000 Training Loss: 0.05937481299042702
Epoch 29845/30000 Training Loss: 0.07628358155488968
Epoch 29846/30000 Training Loss: 0.07363473623991013
Epoch 29847/30000 Training Loss: 0.07992953807115555
Epoch 29848/30000 Training Loss: 0.06606351584196091
Epoch 29849/30000 Training Loss: 0.07115941494703293
Epoch 29850/30000 Training Loss: 0.07700146734714508
Epoch 29850/30000 Validation Loss: 0.0768895

Epoch 29975/30000 Training Loss: 0.07692773640155792
Epoch 29976/30000 Training Loss: 0.08890943974256516
Epoch 29977/30000 Training Loss: 0.06429499387741089
Epoch 29978/30000 Training Loss: 0.07122113555669785
Epoch 29979/30000 Training Loss: 0.06363467127084732
Epoch 29980/30000 Training Loss: 0.07840152829885483
Epoch 29980/30000 Validation Loss: 0.06785768270492554
Epoch 29981/30000 Training Loss: 0.07128793001174927
Epoch 29982/30000 Training Loss: 0.06643220782279968
Epoch 29983/30000 Training Loss: 0.09301088005304337
Epoch 29984/30000 Training Loss: 0.07620496302843094
Epoch 29985/30000 Training Loss: 0.0679331123828888
Epoch 29986/30000 Training Loss: 0.06313331425189972
Epoch 29987/30000 Training Loss: 0.07106880098581314
Epoch 29988/30000 Training Loss: 0.09112560749053955
Epoch 29989/30000 Training Loss: 0.06787580996751785
Epoch 29990/30000 Training Loss: 0.06422186642885208
Epoch 29990/30000 Validation Loss: 0.07221904397010803
Epoch 29991/30000 Training Loss: 0.06422376

In [ ]:
# plot(train_loss, val_loss)

In [ ]:
# param_names = []
# for i, (name, param) in enumerate(model.named_parameters()):
#     param_names.append(name)
#     print(f'{i}: {name} {param.requires_grad}')

In [55]:
best_model_fp = f'models/0308/model_best_epoch_23930_min_val_loss_0.02459999918937683.pkl'
with open(best_model_fp, 'rb') as handle:
    best_model = pickle.load(handle)

# Generating sequences

In [90]:
diffusion_2000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 2000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

In [97]:
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_2000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128, 
                                                           show_intermediate_results=False)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["my lord, i claim your gift, my due by promise, for which your honour and your faith is pawn'd, the earldom of hereford and the moveables the which you promised i should possess.", 'i am a bastard too, i love bastards i am a bastard begot, bastard instructed, bastard in mind, bastard in valour, in every thing illegitimate. one bear will not bite another,'] ['stanley, look to your wife, if she convey letters to richmond, you shall answer it.', "and wherefore should one bastard? take heed, the quarrel's most ominous to us if the son of a whore fight for a whore, he tempts judgment farewell, bastard."]
RAM used: 6068.92 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 4
})
RAM used: 6068.92 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/4 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 4
})
### tokenized_datasets...example [2, 105, 203, 10, 31, 2385, 135, 1820, 10, 105, 2280, 193, 1823, 10, 115, 260, 135, 487, 85, 135, 676, 121, 3876, 9, 26, 10, 78, 11495, 94, 3787, 85, 78, 11788, 78, 260, 89, 3808, 31, 305, 1872, 12, 3]
RAM used: 6068.98 MB


merge and mask:   0%|          | 0/4 [00:00<?, ? examples/s]

RAM used: 6068.98 MB


padding:   0%|          | 0/4 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 4
}) padded dataset
RAM used: 6069.00 MB
RAM used: 6069.00 MB
### End of reading iteration...


  0%|          | 0/2000 [00:00<?, ?it/s]

Generating 20 sentences takes 5 minutes

In [98]:
word_lst_source

["[CLS] my lord, i claim your gift, my due by promise, for which your honour and your faith is pawn'd, the earldom of hereford and the moveables the which you promised i should possess. [SEP] [SEP]",
 '[CLS] i am a bastard too, i love bastards i am a bastard begot, bastard instructed, bastard in mind, bastard in valour, in every thing illegitimate. one bear will not bite another, [SEP] [SEP]',
 "[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
 '[CLS] call her forth to me [SEP] [SEP]']

In [99]:
word_lst_recover

['[CLS] so your to be him king in? [SEP]',
 '[CLS] by sorry well to be to to, or to the good. [SEP]',
 '[CLS] for your one to a christian. [SEP]',
 '[CLS] how hark! [SEP]']